# W_GL_ACCOUNT_DS ETL Process
### Oracle EBS GL_CODE_COMBINATIONS to EDW W_GL_ACCOUNT_DS (Staging)

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2624719 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Extract GL_CODE_COMBINATIONS with segment extraction logic
CREATE OR REPLACE TEMP VIEW stg_code_combinations AS
SELECT
  CAST(gcc.code_combination_id AS STRING) AS gl_account_id,
  gcc.account_type,
  CAST(gcc.chart_of_accounts_id AS STRING) AS chart_of_accounts_id,
  -- Segment mappings (example shown for segments 1-30)
  gcc.segment1, gcc.segment2, gcc.segment3, gcc.segment4, gcc.segment5,
  gcc.segment6, gcc.segment7, gcc.segment8, gcc.segment9, gcc.segment10,
  gcc.segment11, gcc.segment12, gcc.segment13, gcc.segment14, gcc.segment15,
  gcc.segment16, gcc.segment17, gcc.segment18, gcc.segment19, gcc.segment20,
  gcc.segment21, gcc.segment22, gcc.segment23, gcc.segment24, gcc.segment25,
  gcc.segment26, gcc.segment27, gcc.segment28, gcc.segment29, gcc.segment30,
  -- Dynamic segment to column mapping based on flex field definitions
  CASE WHEN ffc.segment_num = 1 THEN gcc.segment1
       WHEN ffc.segment_num = 2 THEN gcc.segment2
       WHEN ffc.segment_num = 3 THEN gcc.segment3
       WHEN ffc.segment_num = 4 THEN gcc.segment4
       WHEN ffc.segment_num = 5 THEN gcc.segment5
       -- Add remaining segments
  END AS segment_value,
  CAST(gcc.created_by AS STRING) AS created_by_id,
  CAST(gcc.last_updated_by AS STRING) AS changed_by_id,
  gcc.creation_date, gcc.last_update_date AS changed_on_dt,
  'N' AS delete_flg,
  p.DATASOURCE_NUM_ID,
  CAST(gcc.code_combination_id AS STRING) AS integration_id,
  p.TENANT_ID, '0' AS x_custom
FROM workspace.oracle_ebs.GL_CODE_COMBINATIONS gcc
CROSS JOIN etl_params p
LEFT JOIN workspace.oracle_ebs.FND_ID_FLEX_SEGMENTS ffc
  ON gcc.chart_of_accounts_id = ffc.id_flex_num
  AND ffc.application_id = 101
  AND ffc.id_flex_code = 'GL#'
WHERE gcc.last_update_date > to_timestamp(p.LAST_EXTRACT_DATE, 'yyyy-MM-dd HH:mm:ss');

In [ ]:
%sql
-- 3) INSERT: Load data into target table W_GL_ACCOUNT_DS
INSERT INTO workspace.oracle_edw.w_gl_account_ds
SELECT 
  gl_account_id, account_type, chart_of_accounts_id,
  segment1, segment2, segment3, segment4, segment5,
  segment6, segment7, segment8, segment9, segment10,
  segment11, segment12, segment13, segment14, segment15,
  segment16, segment17, segment18, segment19, segment20,
  segment21, segment22, segment23, segment24, segment25,
  segment26, segment27, segment28, segment29, segment30,
  created_by_id, changed_by_id, creation_date, changed_on_dt,
  delete_flg, datasource_num_id, integration_id, tenant_id, x_custom
FROM stg_code_combinations;

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION' AS package_name,
    'W_GL_ACCOUNT_DS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 5) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION', 'W_GL_ACCOUNT_DS', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_code_combinations;
DROP VIEW IF EXISTS etl_params;